In [45]:
import os
import pandas as pd

In [46]:
cwd = os.getcwd() # Directorio en el que trabajamos actualmente
BASE_DIR = os.path.dirname(cwd) # Variable BASE_DIR se vuelve el path/directorio actual del archivo
DATA_DIR = os.path.join(BASE_DIR, "files/MLOpsReviews") # DATA_DIR se vuelve el path a la carpeta donde esta alojado los CSV
CACHE_DIR = os.path.join(BASE_DIR, "files/cache") # CACHE_DIR va ser el la carpreta donde alojaremos los CSV procesados
RATING_DIR = os.path.join(BASE_DIR, "files/MLOpsReviews/ratings") # RATING_DIR va ser donde esta ubicado la carpeta de los CSV con rating
working_file = os.path.join(CACHE_DIR, 'stream-data.csv')
working_rating = os.path.join(CACHE_DIR, 'rating-data.csv')
os.makedirs(CACHE_DIR, exist_ok=True)

In [47]:
my_dataframes = []
ratings = []

csv_files = [x for x in os.listdir(DATA_DIR) if x.endswith(".csv")]
for filename in csv_files:
    print(filename)
    platform = filename.replace(".csv", "")
    csv_path = os.path.join(DATA_DIR, filename)
    this_df = pd.read_csv(csv_path, index_col=False)
    this_df['id'] = platform[0]+this_df["show_id"]
    this_df['platform'] = platform
    this_df.drop(columns='show_id', inplace=True)
    print(this_df.head(n=1))
    my_dataframes.append(this_df)

rating_files = [x for x in os.listdir(RATING_DIR) if x.endswith(".csv")]
for filename in rating_files:
    print(filename)
    csv_path = os.path.join(RATING_DIR, filename)
    this_df = pd.read_csv(csv_path, index_col=False)
    print(this_df.head(n=1))
    ratings.append(this_df)

amazon.csv
    type                title      director  \
0  Movie  The Grand Seduction  Don McKellar   

                                             cast country      date_added  \
0  Brendan Gleeson, Taylor Kitsch, Gordon Pinsent  Canada  March 30, 2021   

   release_year rating duration      listed_in  \
0          2014    NaN  113 min  Comedy, Drama   

                                         description   id platform  
0  A small fishing village must procure a local d...  as1   amazon  
disney.csv
    type                                             title  \
0  Movie  Duck the Halls: A Mickey Mouse Christmas Special   

                            director  \
0  Alonso Ramirez Ramos, Dave Wasson   

                                                cast country  \
0  Chris Diamantopoulos, Tony Anselmo, Tress MacN...     NaN   

          date_added  release_year rating duration          listed_in  \
0  November 26, 2021          2016   TV-G   23 min  Animation, Family   

       

In [48]:
all_dataframes = pd.concat(my_dataframes)
all_ratings = pd.concat(ratings)

# /////////////////////////

In [49]:
def removeNulls(dataframe):
    dataframe = dataframe["rating"].fillna('g', inplace=True)
    return dataframe
# Made a function to replace NaN values with the letter 'g' in the 'rating' column

removeNulls(all_dataframes)

In [50]:
def standarize(dataframe):
    dataframe["date_added"] = pd.to_datetime(dataframe["date_added"])
    return dataframe
# Function to standarize date format

standarize(all_dataframes)

,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,id,platform
0,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,2021-03-30,2014,g,113 min,"Comedy, Drama",A small fishing village must procure a local d...,as1,amazon
1,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,2021-03-30,2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,as2,amazon
2,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,2021-03-30,2017,g,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,as3,amazon
3,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,2021-03-30,2014,g,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",as4,amazon
4,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,2021-03-30,1989,g,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,as5,amazon
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8802,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,2019-11-20,2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a...",ns8803,netflix
8803,TV Show,Zombie Dumb,NaN,NaN,NaN,2019-07-01,2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g...",ns8804,netflix
8804,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,2019-11-01,2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...,ns8805,netflix
8805,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,2020-01-11,2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero...",ns8806,netflix


In [51]:
def lowercase(dataframe):
    dataframe = dataframe.apply(lambda x: x.astype(str).str.lower()
                         if (x.dtype == 'object') else x)
    return dataframe
# Function to apply lowercase

all_dataframes = lowercase(all_dataframes)

In [52]:
def durationSplit(dataframe):
    dataframe[['duration_int', 'duration_type']] = dataframe['duration'].str.split(expand=True)
    dataframe['duration_int'] = dataframe['duration_int'].astype(float).astype('Int64')
    #dataframe.drop(columns=["duration"], inplace=True)
    dataframe['duration_type'] = dataframe['duration_type'].replace(['seasons', 'season'], 'season')
    dataframe.drop(columns='duration', inplace=True)
    return dataframe
# Function to split the 'duration' column into two different columns, and standarize the 'seasons' value

all_dataframes = durationSplit(all_dataframes)

In [53]:
mean = all_ratings.groupby("movieId")["rating"].mean()
# all_ratings["rating"] = all_ratings[mean]
all_ratings["rating"] = all_ratings["rating"].groupby([all_ratings["movieId"]]).transform('mean')
all_ratings.drop_duplicates(subset=["movieId"], inplace=True)
all_ratings.drop(columns=['userId','timestamp'], inplace=True)
all_ratings.rename(columns={"rating": "score", "movieId": "id"}, inplace=True)
all_ratings

,score,id
0,3.549127,as680
1,3.556886,ns2186
2,3.604124,hs2381
3,3.547085,ns3663
4,3.532158,as9500
...,...,...
182025,3.470833,ns4069
195443,3.478632,ds619
198037,3.506757,ns7976
240502,3.551829,ns1837


In [54]:
dataset = os.path.join(CACHE_DIR, 'stream-data.csv')
rating_dataset = os.path.join(CACHE_DIR, 'rating-data.csv')
final_dataframe = pd.merge(all_dataframes, all_ratings, on='id',  how="inner")

In [55]:
final_dataframe

,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,id,platform,duration_int,duration_type,score
0,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,as1,amazon,113,min,3.467131
1,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,as2,amazon,110,min,3.548682
2,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,amazon,74,min,3.500000
3,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",as4,amazon,69,min,3.538055
4,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,as5,amazon,45,min,3.478992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22993,movie,zodiac,david fincher,"mark ruffalo, jake gyllenhaal, robert downey j...",united states,2019-11-20,2007,r,"cult movies, dramas, thrillers","a political cartoonist, a crime reporter and a...",ns8803,netflix,158,min,3.438998
22994,tv show,zombie dumb,nan,nan,nan,2019-07-01,2018,tv-y7,"kids' tv, korean tv shows, tv comedies","while living alone in a spooky town, a young g...",ns8804,netflix,2,season,3.515947
22995,movie,zombieland,ruben fleischer,"jesse eisenberg, woody harrelson, emma stone, ...",united states,2019-11-01,2009,r,"comedies, horror movies",looking to survive in a world taken over by zo...,ns8805,netflix,88,min,3.420945
22996,movie,zoom,peter hewitt,"tim allen, courteney cox, chevy chase, kate ma...",united states,2020-01-11,2006,pg,"children & family movies, comedies","dragged from civilian life, a former superhero...",ns8806,netflix,88,min,3.588050


In [56]:
final_dataframe.to_csv(dataset, index=False)

In [57]:
final_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22998 entries, 0 to 22997
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   type           22998 non-null  object        
 1   title          22998 non-null  object        
 2   director       22998 non-null  object        
 3   cast           22998 non-null  object        
 4   country        22998 non-null  object        
 5   date_added     13444 non-null  datetime64[ns]
 6   release_year   22998 non-null  int64         
 7   rating         22998 non-null  object        
 8   listed_in      22998 non-null  object        
 9   description    22998 non-null  object        
 10  id             22998 non-null  object        
 11  platform       22998 non-null  object        
 12  duration_int   22516 non-null  Int64         
 13  duration_type  22516 non-null  object        
 14  score          22998 non-null  float64       
dtypes: Int64(1), dateti

In [58]:
final_dataframe.head()

,type,title,director,cast,country,date_added,release_year,rating,listed_in,description,id,platform,duration_int,duration_type,score
0,movie,the grand seduction,don mckellar,"brendan gleeson, taylor kitsch, gordon pinsent",canada,2021-03-30,2014,g,"comedy, drama",a small fishing village must procure a local d...,as1,amazon,113,min,3.467131
1,movie,take care good night,girish joshi,"mahesh manjrekar, abhay mahajan, sachin khedekar",india,2021-03-30,2018,13+,"drama, international",a metro family decides to fight a cyber crimin...,as2,amazon,110,min,3.548682
2,movie,secrets of deception,josh webber,"tom sizemore, lorenzo lamas, robert lasardo, r...",united states,2021-03-30,2017,g,"action, drama, suspense",after a man discovers his wife is cheating on ...,as3,amazon,74,min,3.500000
3,movie,pink: staying true,sonia anderson,"interviews with: pink, adele, beyoncé, britney...",united states,2021-03-30,2014,g,documentary,"pink breaks the mold once again, bringing her ...",as4,amazon,69,min,3.538055
4,movie,monster maker,giles foster,"harry dean stanton, kieran o'brien, george cos...",united kingdom,2021-03-30,1989,g,"drama, fantasy",teenage matt banting wants to work with a famo...,as5,amazon,45,min,3.478992


In [59]:
plataforma = 'netflix'
year = 2014
duration_type = 'min'
all_dataframes = all_dataframes[all_dataframes["platform"]==plataforma]
all_dataframes = all_dataframes[all_dataframes["release_year"]==year]
all_dataframes = all_dataframes[all_dataframes["duration_type"]==duration_type]
# result = all_dataframes[(all_dataframes['platform']==plataforma) & (all_dataframes['release_year']==year) & (all_dataframes['duration_type']==duration_type)].sort_values(by=["duration_int", "title"]).iloc[-1]

In [60]:
dataset = os.path.join(CACHE_DIR, 'stream-data.csv')

In [61]:
def get_max_duration(anio: int, plataforma: str, dtype: str):
    df = pd.read_csv(dataset)
    df = df[(df['platform']==plataforma) & (df['release_year']==anio) & (df['duration_type']==dtype)].sort_values(by=['duration_int', 'title']).iloc[-1]
    return {'pelicula': df["title"]}

In [62]:
get_max_duration(2019, 'amazon', 'min')

{'pelicula': 'box fan medium  8 hours for sleep'}

In [63]:
def get_score_count(plataforma: str, scored: float, anio: int):
    df = pd.read_csv(dataset)
    df = df[(df['platform']==plataforma) & (df['score']==scored) & (df['release_year']==anio)]
    return {
        'plataforma': plataforma,
        'cantidad': df.count(),
        'anio': anio,
        'score': scored
    }

In [64]:
def get_score_count(plataforma: str, scored: float, anio: int):
    df = pd.read_csv(dataset)
    df = df[(df['platform']==plataforma) & (df['score']>=scored) & (df['release_year']==anio)]
    return {
        'cantidad': len(df),
    }

In [65]:
get_score_count('netflix', 3.6, 2018)

{'cantidad': 114}

In [66]:
df = pd.read_csv(dataset)
df = df[(df['platform']=="netflix") & (df['release_year']==2018)]
df['cast'] = df['cast'].str.split(',')
df = df.explode('cast')
df = df.cast.value_counts()
df

 andrea libman       8
 jigna bharadhwaj    6
 sabina malik        6
 takahiro sakurai    6
 vincent tong        6
                    ..
 hamad ashkanani     1
 noor alghandour     1
 mohammed alalawi    1
 shujoon             1
 revathi pillai      1
Name: cast, Length: 6824, dtype: int64

In [67]:
def get_actor(plataforma: str, anio: int):
    df = pd.read_csv(dataset)
    df = df[(df['platform']==plataforma) & (df['release_year']==anio)]
    df['cast'] = df['cast'].str.split(',')
    df = df.explode('cast')
    return {
        'plataforma': plataforma,
        'anio': anio,
        'actor': df["cast"].value_counts().index.tolist()[0],
        'apariciones': df["cast"].value_counts().values.tolist()[0]
    }



In [68]:
pais = 'canada'
tipo = 'movie'
anio = 2019
df = pd.read_csv(dataset)
df = df[(df['type']==tipo) & (df['country']==pais) & (df['release_year']==anio)]
len(df)


20

In [69]:
get_actor('netflix', 2018)

{'plataforma': 'netflix',
 'anio': 2018,
 'actor': ' andrea libman',
 'apariciones': 8}

In [70]:
df = pd.read_csv(dataset)
df = df.rating.value_counts()
df.head(n=100)

tv-ma        3675
tv-14        3138
r            2154
13+          2117
tv-pg        1654
             ... 
47 min          1
65 min          1
37 min          1
all_ages        1
4 seasons       1
Name: rating, Length: 100, dtype: int64

In [71]:
def get_contents(rating: str):
    df = pd.read_csv(dataset)
    df = df[(df['rating']==rating)]
    return {'rating': rating, 'contenido': len(df)}
get_contents('13+')

{'rating': '13+', 'contenido': 2117}